# 한글폰트 설정 및 라이브러리 임포트

In [ ]:
# 코드 실행 후 재시작
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 필요한 라이브러리 import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 멧플롯립 한글폰트 지정
plt.rc('font', family='NanumBarunGothic')    # 코랩에서 사용
plt.rc('font', family='Malgun Gothic')    # 윈도우에서 사용
plt.rcParams['axes.unicode_minus'] = False

# 빵덕후들의 지도
 - 제주도 서귀포시의 제과점 위치 정보를 제공하는 지도 시각화

## 파일 읽어서 데이터프레임 생성하기

In [ ]:
import pandas as pd
df = pd.read_csv('제주특별자치도_서귀포시_제과점현황_20240401.csv', encoding='cp949')
df.head()

## 데이터 구조 탐색하기

In [ ]:
df.info()

In [ ]:
#'업종명'컬럼의 데이터 확인하기
df['업종명'].value_counts()

In [ ]:
# 결측치 데이터 확인하기
# df.isnull()
df.isnull().sum()

In [ ]:
# 특정 컬럼의 결측치 확인
df[df['소재지(도로명)'].isnull()]
# df.loc[df['소재지(도로명)'].isnull()]

## 데이터 전처리하기

### 이상치 확인하기
- 수치형 값을 갖는 '위도','경도' 컬럼에 이상치가 있는지 boxplot으로 확인

In [ ]:
import matplotlib.pyplot as plt

ax1 = plt.subplot(121) # (1) 첫 번째 공간에 axes 생성
ax1.boxplot(df['위도'])
ax2 = plt.subplot(122)
ax2.boxplot(df['경도'])
plt.show()

### 불필요한 컬럼 삭제하기
- '소재지(도로명)' 컬럼 대신 '소재지(지번)'을 사용하면 되기 때문에 이 컬럼은 삭제한다
- '업종명', '데이터기준일자' 컬럼도 분석에 필요하지 않은 내용으로 보고 삭제한다.

In [ ]:
df.drop(columns=['업종명', '소재지(도로명)', '데이터기준일자'], axis=1, inplace=True)
df.isnull().sum()

In [ ]:
df.head()

### 소재지(지번) 주소에서 읍,면,동 정보 추출하기

In [ ]:
df['소재지(지번)'].str.split(' ').str[2]

In [ ]:
df['읍면동'] = df['소재지(지번)'].str.split(' ').str[2]
df.head()

In [ ]:
df.info()

## 데이터 탐색하기

### 빵집이 제일 많은 동네는 어디일까?

In [ ]:
df['읍면동'].value_counts()

In [ ]:
# 대정읍-> 안덕면 -> 성산읍 순서로 빵집이 많다.

In [ ]:
df.loc[df['읍면동']=='대정읍'][['업소명', '소재지(지번)']]

### 중문동/서호동에 있는 빵집의 주소 알아보기

In [ ]:
# df.loc[df['읍면동']=='중문동'][['업소명', '소재지(지번)']]
df.loc[df['읍면동']=='서호동'][['업소명', '소재지(지번)']]

## 데이터 시각화 하기
- 지도위에 위치와 정보 표시하기
- folium 사용법 <br>
https://python-visualization.github.io/folium/quickstart.html#Getting-Started

### folium사용하기

In [ ]:
!pip install folium

In [ ]:
import folium

In [ ]:
folium.__version__

In [ ]:
# 초기위치(위도,경도)=서귀포시 33.2541205,126.560076
lati, long  = 33.2541205,126.560076
map = folium.Map(location = [lati, long], zoom_start= 10)
map

In [ ]:
map = folium.Map(location = [ lati, long ],
                 zoom_start= 10
                 )
tooltip = 'click'
folium.Marker(
    location = [lati, long ],
    popup = '<strong>hello jeju</strong>',
    icon = folium.Icon(color = 'pink', icon = 'heart'),
    tooltip = tooltip
).add_to(map)
map

### 제과점 위치 마커 표시하기

In [ ]:
df.head()

In [ ]:
df.loc[0,'위도']
df.loc[5, '위도']


### 마커에 정보 넣기

In [ ]:
for idx in df.index :
  lat = df.loc[idx,'위도']
  lng = df.loc[idx,'경도']
  marker = folium.Marker(
      location = [lat, lng],
      popup = f"<div style='width:100px'> <strong>{df.loc[idx,'업소명']}</strong><br>\
      {df.loc[idx, '소재지(지번)']}</div>",
      icon = folium.Icon(color='orange', icon='cloud')
      )
  marker.add_to(map)

map

In [ ]:
from folium.plugins import MarkerCluster

map = folium.Map(location = [33.3616666,126.5291666 ],
                 zoom_start= 10
                 )

marker_cluster = MarkerCluster().add_to(map)

for idx in df.index :
  lat = df.loc[idx,'위도']
  lng = df.loc[idx,'경도']
  marker = folium.Marker(
      location = [lat, lng],
      popup = f"<div style='width:100px'> <strong>{df.loc[idx,'업소명']}</strong><br>\
      {df.loc[idx, '소재지(지번)']}</div>",
      icon = folium.Icon(color='orange', icon='cloud')
      )
  marker.add_to(marker_cluster)

map
